Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD


### IMAGE PREPROCESSING DONE AT LOCAL SYSTEM AND BROUGHT THE DATA IN SHAPE OF CSV TO COLAB - THE PREPROCESSING IS DONE USING FOLLOWING SCRIPT

#### IMAGE RESIZE AND GRESCALE SCRIPT

In [ ]:
import os
# import numpy as np
from PIL import Image
from pathlib import Path

path = "images/"


# This will resise the Image to 50 x 50.
def image_resize(path,filename):
    load_img_rz = np.array(Image.open(os.path.join(path, filename)).resize((50, 50)))
    Image.fromarray(load_img_rz).save(os.path.join(path, filename))

# This will convert image into greyscale
# Warning This will replace the files
def save_as_greyscale(path,filename):
    im = np.array(Image.open(os.path.join(path, filename)).convert('L'))  # you can pass multiple arguments in single line
    gr_im = Image.fromarray(im).save(os.path.join(path,filename))



for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        file_name_with_path = os.path.join(dirpath, filename)
        bn = os.path.basename(filename)
        save_as_greyscale(dirpath,filename)
        image_resize(dirpath,filename)
        print (file_name_with_path)

#### CONVERTED IMAGES INTO NUMPY ARRAY WITH LABELS FROM DIRECTORY NAME USING FOLLOIWNG SCRIPT

In [ ]:
from numpy import asarray
from PIL import Image
#import pandas as pd
# import numpy as np
# import os

# Path of Images Directory
path = "images/flowers/"

# Creating a blank dataframe
df = pd.DataFrame()

# Creating a blank list
listt = []

# This function will convert image to numpy array and flatten the received array
# Function Arguments : file path and directory name as flower label
def image_to_numpy_flaten(path, fl_label):
    image = Image.open(path)
    numpydata = asarray(image)
    fdata = numpydata.flatten()
    fdata = np.append(fdata, fl_label)
    return fdata


# For loop with Os.walk
# Creating Label Name from directory name
# Calling Function image_to_numpy_flatten

for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        label = dirpath.split("/")
        label = label[-1].split("\\")
        label = label[-1]
        result = image_to_numpy_flaten(os.path.join(dirpath,filename), label)
        listt.append(result)



newvar = pd.DataFrame(listt)
newvar.to_csv('flower_data.csv', mode='a', index=False, header=False)

#### LOADING CSV FILE

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/flower_data.csv', header=None, index_col=False)

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,138,154,167,168,162,156,152,149,147,150,146,141,152,169,169,157,162,162,167,173,171,166,168,175,173,168,169,162,149,156,175,183,179,164,147,142,150,160,166,166,...,124,124,137,169,171,175,179,182,177,166,152,143,138,171,174,164,176,187,185,180,156,146,126,132,181,178,171,156,164,174,175,165,145,133,133,135,133,128,128,daisy
1,225,217,221,227,219,132,63,124,189,183,197,184,208,163,137,184,160,133,126,125,113,117,131,130,139,138,139,143,147,147,141,136,121,139,116,144,109,8,23,60,...,236,204,228,232,240,247,244,241,231,222,233,243,237,222,214,216,230,237,226,211,203,173,139,149,140,107,105,107,104,113,116,107,104,109,145,173,142,137,164,daisy
2,102,87,89,76,93,126,115,113,147,123,133,158,131,156,128,156,148,114,94,100,90,103,138,125,111,108,123,111,107,123,124,119,129,137,126,143,133,132,115,132,...,9,9,18,27,31,22,13,8,12,22,61,142,217,114,97,204,119,44,11,20,6,21,19,19,23,24,25,31,39,35,36,38,37,29,20,20,26,35,35,daisy
3,33,26,28,33,33,36,37,31,43,37,53,35,27,49,43,44,40,57,52,54,55,58,69,59,57,81,72,70,78,72,57,29,23,16,35,57,22,48,73,70,...,9,12,9,6,8,10,14,14,11,12,15,15,13,16,16,16,17,18,19,21,22,19,20,20,19,17,17,19,21,20,18,16,17,21,22,20,18,22,22,daisy
4,17,29,42,42,47,45,32,39,38,45,53,73,71,52,48,48,34,41,26,31,36,29,46,60,62,60,36,29,50,50,27,20,23,32,33,22,16,18,20,17,...,24,24,25,26,27,30,30,29,28,33,38,37,31,30,29,28,27,28,29,31,32,30,32,34,33,30,28,26,26,24,29,33,32,28,26,29,33,31,32,daisy


In [ ]:
data.isnull().sum()


0       0
1       0
2       0
3       0
4       0
       ..
2496    0
2497    0
2498    0
2499    0
2500    0
Length: 2501, dtype: int64

In [ ]:
data.info

<bound method DataFrame.info of       0     1     2     3     4     5     ...  2495  2496  2497  2498  2499   2500
0      138   154   167   168   162   156  ...   133   135   133   128   128  daisy
1      225   217   221   227   219   132  ...   145   173   142   137   164  daisy
2      102    87    89    76    93   126  ...    20    20    26    35    35  daisy
3       33    26    28    33    33    36  ...    22    20    18    22    22  daisy
4       17    29    42    42    47    45  ...    26    29    33    31    32  daisy
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...    ...
4318   203   217   201   197    56    96  ...    23    46   107   133   167  tulip
4319   135   132   124   123   135   147  ...    51    62    79    99   114  tulip
4320    80    79    82    89    90    86  ...    84    83    85    79    79  tulip
4321   181   201   166   153   130   183  ...    33    20    26    21    34  tulip
4322    46    93   101   123   106   140  ...    75    

In [ ]:
data[0].shape # Already Flatten 

(4323,)

DATA Shuffling

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,246,231,233,239,248,231,247,237,236,236,238,244,248,250,250,251,245,252,242,229,238,246,243,241,238,239,239,240,240,240,239,239,238,238,239,240,241,242,243,244,...,239,255,251,235,228,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,244,246,246,246,246,246,246,246,246,222,255,dandelion
1,93,87,81,55,50,81,87,71,75,59,76,98,66,54,45,106,143,172,129,84,124,169,167,163,175,153,140,174,177,175,212,213,217,232,241,255,235,218,229,203,...,50,59,64,58,43,33,31,31,32,30,26,27,31,39,46,61,53,59,32,34,45,101,123,134,199,237,253,248,249,249,238,207,181,128,72,66,73,75,93,sunflower
2,134,136,138,147,154,153,169,202,206,134,62,35,36,36,33,30,38,32,64,150,144,138,143,141,176,185,179,177,175,233,228,203,212,227,213,85,94,102,89,94,...,50,135,98,110,138,123,126,130,171,231,201,190,195,202,198,157,162,172,213,233,230,238,222,222,208,213,144,106,41,29,30,30,28,24,22,22,23,21,20,sunflower
3,150,152,165,157,176,167,86,81,117,206,151,111,131,138,156,129,123,119,111,120,116,114,124,110,100,118,116,107,122,151,177,195,189,144,112,101,109,106,95,118,...,69,47,54,61,83,88,85,91,102,84,81,48,53,71,112,137,137,135,132,126,126,116,108,128,128,125,90,72,45,59,80,118,107,110,79,46,33,47,48,sunflower
4,40,37,18,4,6,6,4,8,21,19,24,0,14,17,0,52,104,124,129,105,75,60,51,43,29,21,10,2,0,1,3,4,13,18,24,20,31,33,21,24,...,247,249,244,235,157,138,130,177,145,134,114,74,78,60,65,73,163,255,250,248,241,241,246,239,223,190,194,234,235,220,213,122,96,87,78,79,89,90,90,tulip


In [ ]:
# Separating Input and Output Colmumns 
X = data.iloc[:, :-1].values  # Input Columns
Y = data.iloc[:, -1:].values # Output Columns 

In [ ]:
X.shape

(4323, 2500)

In [ ]:
Y.shape

(4323, 1)

In [ ]:
# Total Imagves 4323 - Data Split
# 4323	60%	2594 Training
# 4323	20%	864 Validataion
# 4323	20%	865 Test

X.shape
x_train = X[:2594,] # 60% Training 
x_test = X[2594:3458,] # 20% Test 
x_val = X[3458:,] # 20% Validatation

y_train = Y[:2594,] # 50% Training
y_test = Y[2594:3458,] # 30% Test
y_val = Y[3458:,] # 20% Validataion




In [ ]:
print (x_train.shape, x_test.shape, x_val.shape)
print (y_train.shape, y_test.shape, y_val.shape)

(2594, 2500) (864, 2500) (865, 2500)
(2594, 1) (864, 1) (865, 1)


# Normalization 

In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_val = x_val.astype('float32') / 255


In [ ]:
x_train # After Normalization 

array([[0.9647059 , 0.90588236, 0.9137255 , ..., 0.9647059 , 0.87058824,
        1.        ],
       [0.3647059 , 0.34117648, 0.31764707, ..., 0.28627452, 0.29411766,
        0.3647059 ],
       [0.5254902 , 0.53333336, 0.5411765 , ..., 0.09019608, 0.08235294,
        0.07843138],
       ...,
       [0.20784314, 0.36078432, 0.47843137, ..., 0.16862746, 0.11372549,
        0.11372549],
       [0.5176471 , 0.4509804 , 0.38431373, ..., 0.4392157 , 0.41568628,
        0.40784314],
       [0.3764706 , 0.4       , 0.41568628, ..., 0.28235295, 0.24705882,
        0.24705882]], dtype=float32)

#### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)
y_val = label_encoder.fit_transform(y_val)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

#### CREATING ARCHITECTURE

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(50 * 50,)))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy',
                metrics=['accuracy'])

#### Training

In [ ]:
history = model.fit(x_train, y_train,epochs=75, validation_data =(x_val, y_val))



Epoch 1/75
82/82 [==============================] - 2s 19ms/step - loss: 4.4283 - accuracy: 0.2067 - val_loss: 2.1853 - val_accuracy: 0.2000
Epoch 2/75
82/82 [==============================] - 1s 17ms/step - loss: 1.8569 - accuracy: 0.2644 - val_loss: 4.6058 - val_accuracy: 0.2405
Epoch 3/75
82/82 [==============================] - 1s 17ms/step - loss: 1.8702 - accuracy: 0.2889 - val_loss: 2.6947 - val_accuracy: 0.1908
Epoch 4/75
82/82 [==============================] - 1s 17ms/step - loss: 1.7069 - accuracy: 0.2745 - val_loss: 2.5705 - val_accuracy: 0.2428
Epoch 5/75
82/82 [==============================] - 1s 18ms/step - loss: 1.6127 - accuracy: 0.3171 - val_loss: 2.3285 - val_accuracy: 0.2486
Epoch 6/75
82/82 [==============================] - 1s 17ms/step - loss: 1.5818 - accuracy: 0.3060 - val_loss: 1.8471 - val_accuracy: 0.2705
Epoch 7/75
82/82 [==============================] - 1s 17ms/step - loss: 1.5538 - accuracy: 0.3258 - val_loss: 2.0808 - val_accuracy: 0.1884
Epoch 8/75
82

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('test_acc:', test_acc)

27/27 [==============================] - 0s 4ms/step - loss: 3.4799 - accuracy: 0.3160
test_acc: 0.3159722089767456


In [ ]:
pred = model.predict(x_test[0:5])
pred

array([[7.35557824e-03, 9.92487192e-01, 1.95319361e-09, 1.57296789e-04,
        2.21961532e-10],
       [1.03031196e-01, 4.75140274e-01, 2.74503410e-01, 8.13016668e-02,
        6.60235286e-02],
       [5.02984412e-02, 9.16827172e-02, 4.04689282e-01, 1.64492026e-01,
        2.88837522e-01],
       [4.48615365e-02, 5.94802462e-02, 8.38501602e-02, 9.77985375e-03,
        8.02028239e-01],
       [5.16461544e-02, 4.45622712e-01, 3.15080583e-01, 1.14313029e-01,
        7.33374208e-02]], dtype=float32)